<a href="https://colab.research.google.com/github/itzraf-lab/Animagine-XL-3/blob/main/Copy_of_Animagine_XL_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Library**

In [ ]:
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors

### **Run pipeline**

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler, AutoencoderKL

lora_model_id = "Linaqruf/style-enhancer-xl-lora"
lora_filename = "style-enhancer-xl.safetensors"

vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)

model = "cagliostrolab/animagine-xl-3.0"

pipe = StableDiffusionXLPipeline.from_pretrained(
    model,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    )
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to('cuda')

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.26.3",
  "_name_or_path": "cagliostrolab/animagine-xl-3.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

### **Generate**

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

pipe.load_lora_weights(lora_model_id, weight_name=lora_filename)
pipe.fuse_lora(lora_scale=0.6)

prompt = "cute, masterpiece, best quality, high quality, portrait, "
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, extra digits, fewer digits, bad eyes"
output = "/content/anime_girl.png"

image = pipe(
    prompt,
    negative_prompt=negative_prompt,
    width=896,
    height=1152,
    guidance_scale=7,
    vae=AutoencoderKL,
    target_size=(896,1152),
    original_size=(3584,4608),
    num_inference_steps=28
    ).images[0]

pipe.unfuse_lora()
image.save(output)
image = Image.open(output)
plt.imshow(image)
plt.axis('off') # to hide the axis

The current API is supported for operating with a single LoRA file. You are trying to load and fuse more than one LoRA which is not well-supported.


OutOfMemoryError: CUDA out of memory. Tried to allocate 114.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 5.06 MiB is free. Process 3307 has 14.74 GiB memory in use. Of the allocated memory 13.72 GiB is allocated by PyTorch, and 901.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### **Clean Memory**

In [ ]:
import gc

if "pipe" in globals():
    del pipe
if "image" in globals():
    del image
torch.cuda.empty_cache()
gc.collect()